In [11]:
#IMPORTING EXTERNAL JUPYTER NOTEBOOKS
import nbimporter
from WWW_classifier_experiments_features import *
from extract_save_features import *
from web_markup_features import *
# import nltk_linguistic_features as nltk_linguis_features
import utils as utils
from importlib import reload
import sys

# reload(sys.modules['readability'])
# reload(sys.modules['nltk_linguistic_features'])
reload(sys.modules['utils'])
reload(sys.modules['step4_WWW_classifier_experiments_features']) # WWW_Classifier_Experiments
reload(sys.modules['extract_save_features'])
reload(sys.modules['step1_web_markup_features'])

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from WWW_classifier_experiments_features.ipynb
Importing Jupyter notebook from extract_save_features.ipynb
Importing Jupyter notebook from web_markup_features.ipynb


<module 'web_markup_features' from 'web_markup_features.ipynb'>

In [12]:
#Applying classifier over the test dataset using invariant features. To classifiers SGD and RandomForest. They needs float data
def test_classifier_floatdata(dir_file_test_fake, dir_file_test_msm, classifier, typefeatures, type_text, filter_features):
    X_fakenews, fakenews_labels_list, fakenews_list_urls = extract_features_filter(typefeatures, type_text, dir_file_test_fake, 1, filter_features ) #1 means fake news. 
    X_msmnews, msmnews_labels_list, msmnews_list_urls = extract_features_filter(typefeatures, type_text, dir_file_test_msm, 0, filter_features)#0 means true news.
    X_temp = np.asarray(X_fakenews+ X_msmnews)
    new_X = []
    for i in X_temp:
        desired_array =  i.astype(np.float) #convert string  arrays to float arrays
        new_X.append(desired_array)
    X = new_X
    y_true = np.asarray(fakenews_labels_list + msmnews_labels_list) 
    y_pred = classifier.predict(X) 
    return [y_true, y_pred]

#Applying classifier over the test dataset using invariant features
def test_classifier_domain(dir_file_test_fake, dir_file_test_msm, classifier, typefeatures, type_text, filter_features):
    X_fakenews, fakenews_labels_list, fakenews_list_urls = extract_features_filter(typefeatures, type_text, dir_file_test_fake, 1, filter_features ) #1 means fake news. 
    X_msmnews, msmnews_labels_list, msmnews_list_urls = extract_features_filter(typefeatures, type_text, dir_file_test_msm, 0, filter_features)#0 means true news.
    X = np.asarray(X_fakenews+ X_msmnews)
    #print (X)
    y_true = np.asarray(fakenews_labels_list + msmnews_labels_list) 
    y_pred = classifier.predict(X) 
    return [y_true, y_pred]

#compute metrics
def compute_metrics_avg(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted') 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    #print ("Accuracy: " + str(accuracy) + " \n Precision: " + str(precision) + " \n Recall: " + str(recall) +  " \n F1_score: " + str(f1) + " \n Confusion Matrix(tn, fp, fn, tp): " + str(tn) +", "+ str(fp) +", "+ str(fn) +", "+ str(tp) + " \n")
    return [accuracy, precision, recall, f1]
    
def compute_metrics(array_metrics):
    return compute_metrics_avg(array_metrics[0], array_metrics[1])

#Applying classifier over the test dataset using invariant features
def classifier_noGT(dir_file_test_data, classifier, typefeatures, type_text, filter_features):
    X_temp, labels_list, list_urls = extract_features_filter(typefeatures, type_text, dir_file_test_data, 1, filter_features ) 
    X = np.asarray(X_temp)
    y_pred = classifier.predict(X) 
    return [list_urls, y_pred]


## Running TAG Classifier

### With ground-truth

In [13]:

# Function run_classifier: this function runs the classifier.
# variables:
# typeFeatures could be: 'tagF'=TopicAgnostic features, 'linguisticF'=Linguistic features, 'webmarkupF'=Webmarkup features
# trainTestData = [[train_fake_files], [train_msm_files], [test_fake_files], [test_msm_files]]
def run_classifier(typeFeatures, type_text, path_data, trainTestData, filter_features):
    train_fake_files = trainTestData[0]
    train_msm_files = trainTestData[1]
    test_fake_files = trainTestData[2]
    test_msm_files = trainTestData[3]
    
    mean_results_knn = [0,0,0,0]
    mean_results_svm = [0,0,0,0]
    mean_results_sgd = [0,0,0,0]
    mean_results_rf = [0,0,0,0]

    total_datasets = len(train_fake_files)
    for index in range(total_datasets):
        #Content Based approach...
        #Getting features vector from dataset
        X_fakenews, fakenews_labels_list, fakenews_list_urls = extract_features_filter(typeFeatures, type_text, path_data+ train_fake_files[index], 1, filter_features ) #1 means fake news. 
        X_msmnews, msmnews_labels_list, msmnews_list_urls = extract_features_filter(typeFeatures, type_text, path_data+ train_msm_files[index], 0, filter_features)#0 means true news.
        X = np.asarray(X_fakenews+ X_msmnews) 
        Y = np.asarray(fakenews_labels_list + msmnews_labels_list)

        #Training classifier
#         print('\nK-Nearest Neighbor:')
        knn_classifier_sy = KNeighborsClassifier(n_neighbors=20)
        knn_classifier_sy.fit(X,Y)#training
#         print('\nSupport Vector Machine:')
        svm_classifier_sy = svm.SVC(kernel = 'linear', C=0.001)#kernel = 'linear'
        svm_classifier_sy.fit(X,Y)#training

#         print('\nStochastic Gradient Descent:')
        sgd_classifier_sy = SGDClassifier(loss="log", penalty="elasticnet")
        sgd_classifier_sy.fit(X,Y)
#         print('\nRandom Forest:')
        rf_classifier_sy = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)#max_depth=2, random_state=0)
        rf_classifier_sy.fit(X,Y)

        #test dataset 
        dir_file_test_fake =path_data+ test_fake_files[index]
        dir_file_test_msm =path_data+ test_msm_files[index]
        measures_knn = compute_metrics(test_classifier_domain(dir_file_test_fake, dir_file_test_msm, knn_classifier_sy, typeFeatures, type_text, filter_features))
        measures_svm = compute_metrics(test_classifier_domain(dir_file_test_fake, dir_file_test_msm, svm_classifier_sy, typeFeatures, type_text, filter_features))
        measures_sgd = compute_metrics(test_classifier_floatdata(dir_file_test_fake, dir_file_test_msm, sgd_classifier_sy, typeFeatures, type_text, filter_features))
        measures_rf = compute_metrics(test_classifier_floatdata(dir_file_test_fake, dir_file_test_msm, rf_classifier_sy, typeFeatures, type_text, filter_features))

        number_measures = 4
        for j in range(number_measures):
            mean_results_knn[j] = mean_results_knn[j] + measures_knn[j]
            mean_results_svm[j] = mean_results_svm[j] + measures_svm[j]
            mean_results_sgd[j] = mean_results_sgd[j] + measures_sgd[j]
            mean_results_rf[j] = mean_results_rf[j] + measures_rf[j]
    
    print ("Accuracy KNN: " + str(mean_results_knn[0]/total_datasets) )
    print ("Accuracy SVM: " + str(mean_results_svm[0]/total_datasets)  )  
    print ("Accuracy SDG: " + str(mean_results_sgd[0]/total_datasets) )
    print ("Accuracy RF: " + str(mean_results_rf[0]/total_datasets)  )

### Without ground-truth

In [14]:

# Function run_classifier: this function runs the classifier without a ground truth.
# variables:
# typeFeatures could be: 'tagF'=TopicAgnostic features, 'linguisticF'=Linguistic features, 'webmarkupF'=Webmarkup features
# trainTestData = [[train_fake_files], [train_msm_files], [test_fake_files], [test_msm_files]]
def run_classifier_noGT(typeFeatures, type_text, path_data, trainTestData, filter_features):
    train_fake_files = trainTestData[0]
    train_msm_files = trainTestData[1]
    test_data = trainTestData[2]
    
    total_datasets = len(train_fake_files)
    for index in range(total_datasets):
        #Content Based approach...
        #Getting features vector from dataset
        X_fakenews, fakenews_labels_list, fakenews_list_urls = extract_features_filter(typeFeatures, type_text, path_data+ train_fake_files[index], 1, filter_features ) #1 means fake news. 
        X_msmnews, msmnews_labels_list, msmnews_list_urls = extract_features_filter(typeFeatures, type_text, path_data+ train_msm_files[index], 0, filter_features)#0 means true news.
        X = np.asarray(X_fakenews+ X_msmnews) 
        Y = np.asarray(fakenews_labels_list + msmnews_labels_list)

        #Training classifier
        svm_classifier_sy = svm.SVC(kernel = 'linear', C=0.001)#kernel = 'linear'
        svm_classifier_sy.fit(X,Y)#training

        #test dataset 
        dir_file_test_data =path_data+ test_data[index]
        output_svm = classifier_noGT(dir_file_test_data, svm_classifier_sy, typeFeatures, type_text, filter_features)
        print('CLASSIFIER RESULTS: ')
        print ("list_urls")
        print (output_svm[0])
        print ("y_pred")
        print (output_svm[1])


In [15]:
#Feature that will be filtered. The arrays contain the feature's indexes. Those features were selected after feature engeeniering
#HEADLINES
h_liwc_filter = [85, 55, 74, 77]
h_readability_filter = [7, 18]
h_nlkt_filter = [3, 4, 6, 16, 21, 24, 25, 28, 31, 32]
headline_filter_features = [h_liwc_filter, h_readability_filter, h_nlkt_filter]

#CONTENT
c_liwc_filter = [23, 24, 25, 26, 27, 28]
c_readability_filter = []
c_nlkt_filter = []
content_filter_features = [c_liwc_filter, c_readability_filter, c_nlkt_filter]

#HEAD_CONTENT
hc_liwc_filter = [23, 24, 25, 26, 27, 28]
hc_readability_filter = []
hc_nlkt_filter = [2, 7, 13, 14, 18, 19, 22, 32]
headline_content_filter_features = [hc_liwc_filter, hc_readability_filter, hc_nlkt_filter]



### Extract Features

In [36]:
url = 'https://www.nbcnews.com/politics/2020-election/dems-alter-how-they-run-campaigns-prove-liberal-values-dodge-n984576'
output_file_name = '/Users/.../Fakenews/paper_fn/Experiments/data/discovery_paper/test_discoveryCrawlData.csv'

get_WebFeatures_from_url(url, output_file_name)
#extract LIWIC features manually (it is optional)
get_save_Readability_NLTK_features(output_file_name, 2)# 2 = content



https://www.nbcnews.com/politics/2020-election/dems-alter-how-they-run-campaigns-prove-liberal-values-dodge-n984576
article--
Writing... 1
[Warning] LIWIC features were not extracted. Please extract those manually and update the output cvs file. Remenber this is just optional.


'completed'

In [103]:
path_directory = '/Users/.../Fakenews/paper_fn/Experiments/data/discovery_paper/testjson/'
output_file_name = '/Users/.../Fakenews/paper_fn/Experiments/data/discovery_paper/testjson/testjson.csv'

get_WebFeatures_from_directory_json(path_directory, output_file_name)
#extract LIWIC features manually (it is optional)
get_save_Readability_NLTK_features(output_file_name, 1)# 2 = content

testjson.csv
infowarsnews_sample.html
testjsonfile1.json
2, 
done
[Warning] LIWIC features were not extracted. Please extract those manually and update the output cvs file. Remenber this is just optional.


'completed'

### Classification without ground Truth (noGT)

In [104]:
path_data = "/Users/.../Fakenews/paper_fn/Experiments/data/domain_agnostic/"

train_fake_files = ['LIWC2015_fakenews_celebrity_content.csv'] #'LIWC2015_fakenews_celebrity.csv','LIWC2015_fakenews_celebrity_content.csv','LIWC2015_fakenews_celebrity_headContent.csv']
train_msm_files = ['LIWC2015_realnews_celebrity_content.csv'] #'LIWC2015_realnews_celebrity.csv' , 'LIWC2015_realnews_celebrity_content.csv', 'LIWC2015_realnews_celebrity_headContent.csv']
dir_file_test_data = ['testjson.csv']

features = [['linguisticF_NLTK',  'readabilityF','webmarkupF'], []]
typeFeatures = features[0]
type_text = 2
filter_features = content_filter_features
run_classifier_noGT(typeFeatures, type_text, path_data, [train_fake_files, train_msm_files, dir_file_test_data], filter_features)


CLASSIFIER RESULTS: 
list_urls
['https://www.infowars.com/trump-put-more-women-in-top-roles-than-obama-bush-clinton/']
y_pred
[0]


### Effectiveness for Different Topics


In [12]:
path_data = "/Users/.../Fakenews/paper_fn/Experiments/data/domain_agnostic/"

train_fake_files = ['LIWC2015_fakenews_NYUcrawl_headContent.csv'] #'LIWC2015_fakenews_celebrity.csv','LIWC2015_fakenews_celebrity_content.csv','LIWC2015_fakenews_celebrity_headContent.csv']
train_msm_files = ['LIWC2015_realnews_NYUcrawl_headContent.csv'] #'LIWC2015_realnews_celebrity.csv' , 'LIWC2015_realnews_celebrity_content.csv', 'LIWC2015_realnews_celebrity_headContent.csv']
test_files = ['LIWC2015_fakenews_celebrity_content.csv']#'LIWC2015_fakenews_political2016.csv' ,'LIWC2015_fakenews_political2016_content.csv','LIWC2015_fakenews_political2016_headContent.csv']

features = [['linguisticF_NLTK',  'readabilityF','webmarkupF'], []]
typeFeatures = features[0]
type_text = 2
filter_features = content_filter_features
run_classifier_noGT(typeFeatures, type_text, path_data, [train_fake_files, train_msm_files, test_files], filter_features)


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Accuracy KNN: 0.6929955290611028
Accuracy SVM: 0.7660208643815202
Accuracy SDG: 0.7287630402384501
Accuracy RF: 0.7496274217585693


In [25]:
path_data = "/Users/.../Fakenews/paper_fn/Experiments/data/domain_agnostic/"

test_fake_files = ['LIWC2015_fakenews_celebrity_content.csv'] #'LIWC2015_fakenews_celebrity.csv','LIWC2015_fakenews_celebrity_content.csv','LIWC2015_fakenews_celebrity_headContent.csv']
test_msm_files = ['LIWC2015_realnews_celebrity_content.csv'] #'LIWC2015_realnews_celebrity.csv' , 'LIWC2015_realnews_celebrity_content.csv', 'LIWC2015_realnews_celebrity_headContent.csv']
train_fake_files = ['LIWC2015_fakenews_political2016_content.csv']#'LIWC2015_fakenews_political2016.csv' ,'LIWC2015_fakenews_political2016_content.csv','LIWC2015_fakenews_political2016_headContent.csv']
train_msm_files = ['LIWC2015_realnews_political2016_content.csv'] #'LIWC2015_realnews_political2016.csv' , 'LIWC2015_realnews_political2016_content.csv', 'LIWC2015_realnews_political2016_headContent.csv']

features = [['linguisticF_NLTK', 'linguisticF_LIWC',  'readabilityF','webmarkupF']]
typeFeatures = features[0]
type_text = 2
filter_features = content_filter_features
run_classifier(typeFeatures, type_text, path_data, [train_fake_files, train_msm_files, test_fake_files, test_msm_files], filter_features)



/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Accuracy KNN: 0.603448275862069
Accuracy SVM: 0.6336206896551724
Accuracy SDG: 0.5517241379310345
Accuracy RF: 0.603448275862069
